## Exercício 2

#### Marcos Cesar Ribeiro de Camargo - 9278045
#### Rafael Augusto Monteiro - 9293095


In [1]:
import numpy as np

In [150]:
class MLP(object):
    model = None

    @staticmethod
    def f(net):
        return ( 1/ (1+ np.exp(-net)) )

    @staticmethod
    def df_dnet(f_net):
        return ( f_net * (1 - f_net) )

    def __init__(self, input_length=2, hidden_length=[3, 3], output_length=1, activation_function=f , d_activation_function=df_dnet):
        hidden_layer = list()
        previous_length = input_length
        for layer_length in hidden_length:
            hidden_layer.append(np.random.rand(layer_length, previous_length+1) - 0.5)
            previous_length = layer_length

        self.model = {
            'input_length': input_length, 
            'hidden_length': hidden_length, 
            'output_length': output_length, 
            'activation_function': activation_function.__func__, 
            'd_activation_function': d_activation_function.__func__,
            'hidden': hidden_layer,
            'output': (np.random.rand(output_length, previous_length+1) - 0.5),
        }

    def forward(self, x):
        # Recuperando valores do modelo
        hidden = self.model['hidden']
        hidden_length = len(self.model['hidden_length'])
        output = self.model['output']
        f = self.model['activation_function']
        df = self.model['d_activation_function']

        # Camadas Escondidas
        net_h = [None] * hidden_length
        f_net_h = [None] * hidden_length
        df_net_h = [None] * hidden_length
        
        
        # Adicionando 1 para multiplicar o Theta.
        previous_layer = np.pad(x, (0, 1), 'constant', constant_values=(1))
        for i in range(hidden_length):
            net_h[i] = np.sum(np.multiply(hidden[i], previous_layer), axis=1)
            f_net_h[i] = f(net_h[i])
            df_net_h[i] = df(f_net_h[i])
            previous_layer = np.pad(f_net_h[i], (0, 1), 'constant', constant_values=(1))

            
        # Camada de Saída
        net_o = np.sum(np.multiply(output, previous_layer), axis=1)
        f_net_o = f(net_o)
        df_net_o = df(f_net_o)

        # Retornando valores do forward.
        return {
            'net_h': net_h,
            'f_net_h': f_net_h,
            'df_net_h': df_net_h,
            'net_o': net_o,
            'f_net_o': f_net_o,
            'df_net_o': df_net_o,
        }

    def backpropagation(self, X, Y, eta=0.1, threshold=1e-3, max_inter = 200000):
        squaredError = 2*threshold
        hidden_length = len(self.model['hidden_length'])
        counter = 0

        while(squaredError > threshold and counter < max_inter):
            squaredError = 0
            # Pra cada valor do conjunto de dados
            for x, y in zip(X, Y):
                # Calculando saída
                results = self.forward(x)
                #  Calculando o erro
                error = (y - results['f_net_o'])                
                squaredError += np.sum(np.power(error, 2))
                

                # Backwards camada de saída
                h = hidden_length - 1
                delta_o = error * results['df_net_o']
                f_net_h = np.pad(results['f_net_h'][h], (0, 1), 'constant', constant_values=(1))
                dE2_dw_o = np.multiply(np.array([-2*delta_o]).T, np.array([f_net_h]))
                
                # Backwards camada escondida
                delta_h = [None] * hidden_length
                dE2_dw_h = [None] * hidden_length
                delta = delta_o
                w_o_kj = self.model['output'][:,0:self.model['hidden_length'][h]] 

                # For reverso
                for i in reversed(range(1, hidden_length)):
                    # Cálculo das adaptações
                    delta_h[i] = np.array([results['df_net_h'][i]]) * np.dot(delta, w_o_kj)         
                    dE2_dw_h[i] = np.multiply(-2*delta_h[i].T,
                        np.pad(results['f_net_h'][i-1], (0, 1), 'constant', constant_values=(1)))
                    # Controlando as iterações
                    delta = delta_h[i]
                    w_o_kj = self.model['hidden'][i][:, 0:self.model['hidden_length'][i-1]]
                    
                delta_h[0] = np.array([results['df_net_h'][0]]) * np.dot(delta, w_o_kj)               
                dE2_dw_h[0] =  np.multiply(-2*delta_h[0].T,
                    np.pad(x, (0, 1), 'constant', constant_values=(1)))
                               

                # Aplicar adaptação na saída
                self.model['output'] = self.model['output'] - eta * dE2_dw_o

                # Aplicar adaptação na escondida
                for i in range(hidden_length):
                    self.model['hidden'][i] = self.model['hidden'][i] - eta * dE2_dw_h[i]

            squaredError = squaredError / len(X) 
            counter += 1
            if(counter % 1000 == 0):
                print('error %.6lf - iter. %d' % (squaredError, counter))


xor_x = np.array([0, 0, 0, 1, 1, 0, 1, 1]).reshape((4,2))
xor_y = np.array([x[0]^x[1] for x in xor_x])

mlp = MLP()
mlp.backpropagation(xor_x, xor_y, eta=0.1)

    
for x, y in zip(xor_x, xor_y):
    print(y == round(mlp.forward(x)['f_net_o'][0]))
                

error 0.254563 - iter. 1000
error 0.254252 - iter. 2000
error 0.254027 - iter. 3000
error 0.253862 - iter. 4000
error 0.253738 - iter. 5000
error 0.253644 - iter. 6000
error 0.253570 - iter. 7000
error 0.253513 - iter. 8000
error 0.253467 - iter. 9000
error 0.253429 - iter. 10000
error 0.253399 - iter. 11000
error 0.253373 - iter. 12000
error 0.253351 - iter. 13000
error 0.253331 - iter. 14000
error 0.253313 - iter. 15000
error 0.253296 - iter. 16000
error 0.253276 - iter. 17000
error 0.253251 - iter. 18000
error 0.253212 - iter. 19000
error 0.253137 - iter. 20000
error 0.252953 - iter. 21000
error 0.252352 - iter. 22000
error 0.249067 - iter. 23000
error 0.204367 - iter. 24000
error 0.176549 - iter. 25000
error 0.172382 - iter. 26000
error 0.170889 - iter. 27000
error 0.170148 - iter. 28000
error 0.169710 - iter. 29000
error 0.169421 - iter. 30000
error 0.169215 - iter. 31000
error 0.169058 - iter. 32000
error 0.168930 - iter. 33000
error 0.168820 - iter. 34000
error 0.168719 - iter. 

In [37]:
xor_x = np.array([0, 0, 0, 1, 1, 0, 1, 1]).reshape((4,2))
xor_y = np.array([x[0]^x[1] for x in xor_x])

mlp = MLP()
mlp.backpropagation(xor_x, xor_y, eta=0.5)

for x, y in zip(xor_x, xor_y):
    print(y == round(mlp.forward(x)['f_net_o'][0]))

error 0.256889 - iter. 1000
error 0.016054 - iter. 2000
error 0.002772 - iter. 3000
error 0.001420 - iter. 4000
True
True
True
True


In [156]:
n = 10 # Tamanho da matriz 
X = np.identity(n) # Matriz identidade de entrada
Y = np.identity(n) # Matriz identidade de saída
hidden_length = int(np.log2(n*n)) # Número de neurônios na camada de saída
input_length = X.size # Número de neurônios de entrada
output_length = Y.size # Número de neurônios de saída

# Iniciando modelo da rede MLP
mlp = MLP(input_length=input_length, hidden_length=[hidden_length, hidden_length, hidden_length], output_length=output_length)
# Backpropagation - Treinamento da MLP
mlp.backpropagation([X.flat], [Y.flat], eta=0.3)
# Printando saída da rede, arredondando valores para inteiros.
np.rint(mlp.forward(Y.flat)['f_net_o']).reshape((n,n))

error 0.012318 - iter. 1000
error 0.006092 - iter. 2000
error 0.004042 - iter. 3000
error 0.003024 - iter. 4000
error 0.002414 - iter. 5000
error 0.002009 - iter. 6000
error 0.001721 - iter. 7000
error 0.001504 - iter. 8000
error 0.001336 - iter. 9000
error 0.001202 - iter. 10000
error 0.001092 - iter. 11000
error 0.001001 - iter. 12000


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])